In [16]:
import sqlite3
import hashlib

In [13]:
dbname = ('users.db')
conn = sqlite3.connect(dbname, isolation_level=None)

In [14]:
cursor = conn.cursor()
sql = """CREATE TABLE IF NOT EXISTS userinfo(id, password)"""

cursor.execute(sql)
conn.commit()

sql = """CREATE TABLE IF NOT EXISTS user_stock(id, securities_code, amount, buy_date)"""
cursor.execute(sql)
conn.commit()

In [15]:
sql = """SELECT name FROM sqlite_master WHERE TYPE='table'"""

for t in cursor.execute(sql):
    print(t)

('userinfo',)
('user_stock',)


In [19]:
username = "user"
password = "Password"

password_hash = hashlib.sha256(password.encode()).hexdigest()

In [20]:
sql = """INSERT INTO userinfo VALUES(?, ?)"""

data = ((username, password_hash))
cursor.execute(sql, data)
conn.commit()

In [22]:
sql = """INSERT INTO userinfo VALUES(?,?)"""

data = [
    ("user2", hashlib.sha256("Password2".encode()).hexdigest()),
    ("user3", hashlib.sha256("Password3".encode()).hexdigest()),
    ("user4", hashlib.sha256("Password4".encode()).hexdigest()),
    ("user5", hashlib.sha256("Password5".encode()).hexdigest())
]

cursor.executemany(sql, data)
conn.commit()

In [24]:
sql = """SELECT * FROM userinfo"""
cursor.execute(sql)
print(cursor.fetchall())

[('user', 'e7cf3ef4f17c3999a94f2c6f612e8a888e5b1026878e4e19398b23bd38ec221a'), ('user2', '1be0222750aaf3889ab95b5d593ba12e4ff1046474702d6b4779f4b527305b23'), ('user3', '2538f153f36161c45c3c90afaa3f9ccc5b0fa5554c7c582efe67193abb2d5202'), ('user4', 'db514f5b3285acaa1ad28290f5fefc38f2761a1f297b1d24f8129dd64638825d'), ('user5', '8180d5783fea9f86479e748f6d2d1196c4a8e143643119398c16367d2c3d50f2')]


In [38]:
sql = """SELECT * FROM userinfo WHERE id = ?"""

search_id = 'user'

cursor.execute(sql,[search_id])
print(cursor.fetchall())
print(type(cursor.fetchall()))

[('user', 'e7cf3ef4f17c3999a94f2c6f612e8a888e5b1026878e4e19398b23bd38ec221a')]
<class 'list'>


In [11]:
conn.close()